In [1]:
# Create a simple test DataFrame
import pandas as pd

col1 = [1, 3, 5, 2, 7, 3, 8, 4]
col2 = [3, 2, 4, 1, 7, 6, 8, 5]

df = pd.DataFrame([{'idx': i, 'col1': v1, 'col2': v2} for (i,v1,v2) in zip(range(len(col1)), col1, col2)])
long_df = df.melt(id_vars=['idx'], var_name='var', value_name='val')

In [2]:
# View the plotted DataFrame
import altair as alt
from IPython.display import display

chart = alt.Chart(long_df).mark_line().encode(
    x=alt.X('idx'),
    y=alt.Y('val'),
    color=alt.Color('var'),
)
chart

alt.Chart(...)

In [3]:
# What's the spec look like?
chart.to_dict()

{'config': {'view': {'continuousWidth': 400, 'continuousHeight': 300}},
 'data': {'name': 'data-8fc9dc7a50e0f63ea5494b532b1d8059'},
 'mark': 'line',
 'encoding': {'color': {'type': 'nominal', 'field': 'var'},
  'x': {'type': 'quantitative', 'field': 'idx'},
  'y': {'type': 'quantitative', 'field': 'val'}},
 '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json',
 'datasets': {'data-8fc9dc7a50e0f63ea5494b532b1d8059': [{'idx': 0,
    'var': 'col1',
    'val': 1},
   {'idx': 1, 'var': 'col1', 'val': 3},
   {'idx': 2, 'var': 'col1', 'val': 5},
   {'idx': 3, 'var': 'col1', 'val': 2},
   {'idx': 4, 'var': 'col1', 'val': 7},
   {'idx': 5, 'var': 'col1', 'val': 3},
   {'idx': 6, 'var': 'col1', 'val': 8},
   {'idx': 7, 'var': 'col1', 'val': 4},
   {'idx': 0, 'var': 'col2', 'val': 3},
   {'idx': 1, 'var': 'col2', 'val': 2},
   {'idx': 2, 'var': 'col2', 'val': 4},
   {'idx': 3, 'var': 'col2', 'val': 1},
   {'idx': 4, 'var': 'col2', 'val': 7},
   {'idx': 5, 'var': 'col2', 'val': 6},
   {

In [4]:
# That data is specific but the spec is reusable, can we isolate the spec from the data?
# Yes, define a data transformer
def art_data_transformer(data, url_path='', endpoint='art'):
    
    columns = data['var'].unique()
    uri = f"{url_path}/{endpoint}?params={','.join(columns)}"
    
    return {"url": uri, "format": {"type": "json"}}

alt.data_transformers.register('art', art_data_transformer)

# Altair does lazy spec creation so create and save within the context.
with alt.data_transformers.enable('art', url_path='/some/url/path'):
    spec = chart.to_dict()
spec

{'config': {'view': {'continuousWidth': 400, 'continuousHeight': 300}},
 'data': {'url': '/some/url/path/art?params=col1,col2',
  'format': {'type': 'json'}},
 'mark': 'line',
 'encoding': {'color': {'type': 'nominal', 'field': 'var'},
  'x': {'type': 'quantitative', 'field': 'idx'},
  'y': {'type': 'quantitative', 'field': 'val'}},
 '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json'}

In [5]:
# We can do similar with the themes.  This is a lot more open.
def art_theme(width=600, height=150):
    return {
        'config': {
            'view': {'continuousWidth': width, 'continuousHeight': height,}
        }
    }
alt.themes.register('art', art_theme)

with alt.themes.enable('art', height=200):
    spec = chart.to_dict()
    display(chart)
spec  # Height and width should be different.

alt.Chart(...)

{'config': {'view': {'continuousWidth': 600, 'continuousHeight': 200}},
 'data': {'name': 'data-8fc9dc7a50e0f63ea5494b532b1d8059'},
 'mark': 'line',
 'encoding': {'color': {'type': 'nominal', 'field': 'var'},
  'x': {'type': 'quantitative', 'field': 'idx'},
  'y': {'type': 'quantitative', 'field': 'val'}},
 '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json',
 'datasets': {'data-8fc9dc7a50e0f63ea5494b532b1d8059': [{'idx': 0,
    'var': 'col1',
    'val': 1},
   {'idx': 1, 'var': 'col1', 'val': 3},
   {'idx': 2, 'var': 'col1', 'val': 5},
   {'idx': 3, 'var': 'col1', 'val': 2},
   {'idx': 4, 'var': 'col1', 'val': 7},
   {'idx': 5, 'var': 'col1', 'val': 3},
   {'idx': 6, 'var': 'col1', 'val': 8},
   {'idx': 7, 'var': 'col1', 'val': 4},
   {'idx': 0, 'var': 'col2', 'val': 3},
   {'idx': 1, 'var': 'col2', 'val': 2},
   {'idx': 2, 'var': 'col2', 'val': 4},
   {'idx': 3, 'var': 'col2', 'val': 1},
   {'idx': 4, 'var': 'col2', 'val': 7},
   {'idx': 5, 'var': 'col2', 'val': 6},
   {

In [6]:
# Can also apply both
theme_kwargs = {  # Use dictionary and ** to keep context (with) statement cleaner
    'width': 800,
    'height': 100
}

# Note that since we messed with the data_transformer, the chart won't work.  Try enabling 'json' though.
with alt.themes.enable('art', **theme_kwargs) as theme, alt.data_transformers.enable('art') as data:
    spec = chart.to_dict()
spec

{'config': {'view': {'continuousWidth': 800, 'continuousHeight': 100}},
 'data': {'url': '/art?params=col1,col2', 'format': {'type': 'json'}},
 'mark': 'line',
 'encoding': {'color': {'type': 'nominal', 'field': 'var'},
  'x': {'type': 'quantitative', 'field': 'idx'},
  'y': {'type': 'quantitative', 'field': 'val'}},
 '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json'}

In [7]:
# Altair will also freely pass valid vega-lite data definitions.
# https://vega.github.io/vega-lite/docs/data.html
data = {
    'name': 'some look-up',
}
chart_json_data = alt.Chart(data).mark_line().encode(
    x=alt.X('idx:Q'),
    y=alt.Y('val:Q'),
    color=alt.Color('var:N'),
)
chart_json_data.to_dict()

{'config': {'view': {'continuousWidth': 400, 'continuousHeight': 300}},
 'data': {'name': 'some look-up'},
 'mark': 'line',
 'encoding': {'color': {'type': 'nominal', 'field': 'var'},
  'x': {'type': 'quantitative', 'field': 'idx'},
  'y': {'type': 'quantitative', 'field': 'val'}},
 '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json'}

All the data methods are intended as direction for vega-lite to access the data.  When using the React Vega object you can pass in a different data definition that will override the one in the spec.